# Jupyter Example
## LAMMPS: pour/in.pour
Reference: https://github.com/lammps/lammps/blob/master/examples/pour/in.pour  
Required files: None  
Description: Pour granular particles into chute container, then induce flow

In [1]:
from __future__ import print_function
import random, math
from lammps import IPyLammps

In [2]:
L = IPyLammps()
L.atom_style("sphere")
L.boundary("p p fm")
L.newton("off")
L.comm_modify("vel yes")
L.region("reg block -10 10 -10 10 -0.5 16 units box")
L.create_box("1 reg")
L.neighbor("0.2 bin")
L.neigh_modify("delay 0")

LAMMPS output is captured by PyLammps wrapper


In [3]:
# IMPORTANT NOTE: these values of stiffness (2000) and timestep (0.001)
# are used in this example file to produce a quick simulation and movie.
# More appropriate values for realistic simulations would be
# k = 2.0e5 and dt = 0.0001, as in bench/in.chute.

L.pair_style("gran/hooke/history 2000.0 NULL 50.0 NULL 0.5 0")
L.pair_coeff("* *")

In [4]:
L.fix("1 all nve/sphere")
L.fix("2 all gravity 1.0 spherical 0.0 -180.0")
L.fix("zlower all wall/gran hooke/history 2000.0 NULL 50.0 NULL 0.5 0 zplane 0.0 2000.0")
L.region("slab block -9.0 9.0 -9.0 9.0 10.0 15.0 units box")
L.fix("ins all pour 3000 1 300719 vol 0.13 50 region slab")
L.compute("1 all erotate/sphere")

In [5]:
L.dump("id all atom 1000 dump.pour")

#L.dump("2 all image 1000 image.*.jpg type type axes yes 0.8 0.02 view 80 -30")
#L.dump_modify("2 pad 5")    

L.dump("3 all movie 1000 mov.mp4 type type axes yes 0.8 0.02 view 80 -30")
L.dump_modify("3 pad 5")

In [6]:
L.thermo_style("custom step atoms ke c_1 vol")
L.thermo("1000")
L.thermo_modify("lost ignore norm no")
L.compute_modify("thermo_temp dynamic/dof yes")
L.timestep("0.001")
L.run("25000")

['Neighbor list info ...',
 '  update every 1 steps, delay 0 steps, check yes',
 '  max neighbors/atom: 2000, page size: 100000',
 '  master list distance cutoff = 1.2',
 '  ghost atom cutoff = 1.2',
 '  binsize = 0.6, bins = 34 34 28',
 '  1 neighbor lists, perpetual/occasional/extra = 1 0 0',
 '  (1) pair gran/hooke/history, perpetual',
 '      attributes: half, newton off, size, history',
 '      pair build: half/size/bin/newtoff',
 '      stencil: half/bin/3d/newtoff',
 '      bin: standard',
 'Setting up Verlet run ...',
 '  Unit style    : lj',
 '  Current step  : 0',
 '  Time step     : 0.001',
 'Per MPI rank memory allocation (min/avg/max) = 5.955 | 5.955 | 5.955 Mbytes',
 'Step Atoms KinEng c_1 Volume ',
 '       0        0           -0            0         6600 ',
 'WARNING: Less insertions than requested (../fix_pour.cpp:679)',
 '    1000      720    1153.5788            0         6600 ',
 'WARNING: Less insertions than requested (../fix_pour.cpp:679)',
 'WARNING: Less inser

In [7]:
L.unfix("ins")
L.fix("2 all gravity 1.0 chute 26.0")
L.run("25000")

['Setting up Verlet run ...',
 '  Unit style    : lj',
 '  Current step  : 25000',
 '  Time step     : 0.001',
 'Per MPI rank memory allocation (min/avg/max) = 12.05 | 12.05 | 12.05 Mbytes',
 'Step Atoms KinEng c_1 Volume ',
 '   25000     3000  0.001094739 0.0008512998         6600 ',
 '   26000     3000    42.692829    2.6538187         6600 ',
 '   27000     3000     129.7181    7.0335968         6600 ',
 '   28000     3000    261.46568    12.174857         6600 ',
 '   29000     3000    459.52369    19.161276         6600 ',
 '   30000     3000    734.60428    25.574934         6600 ',
 '   31000     3000    1072.2195    35.742797         6600 ',
 '   32000     3000    1483.4306    39.845576         6600 ',
 '   33000     3000    1933.0487    47.478062         6600 ',
 '   34000     3000    2547.2381    61.889492         6600 ',
 '   35000     3000    3187.6923    60.799558         6600 ',
 '   36000     3000     3875.391    70.682859         6600 ',
 '   37000     3000      4528.4

In [8]:
L.undump("3")
L.video("mov.mp4")